<a href="https://colab.research.google.com/github/darrenmcewan/IntroNLP/blob/main/MovieReview_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and data

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...


Import Libraries

In [ ]:
import nltk
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk import wordpunct_tokenize, word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

ps = PorterStemmer()

sw = stopwords.words('english')

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve

LookupError: ignored

In [ ]:
from collections import defaultdict
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import LabelEncoder

Import Data

In [ ]:
data = pd.read_csv("https://bd.darrenmcewan.com/ratings.tsv", sep='\t')
data.review=data.review.astype(str)
data.head(10)

# Data Cleaning and Split in Train/Test Set

There were 55 rows that were empty. Dropping them leave us with 7945 rows of data to work with

In [ ]:
#Drop blank rows
data['review'].replace('', np.nan, inplace=True)
data.dropna(subset=['review'],inplace=True)
data.dropna(subset=['label'],inplace=True)

#All words to lowercase
data['review'] = [entry.lower() for entry in data['review']]

#tokenization
data['review']= [word_tokenize(entry) for entry in data['review']]

In [ ]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(data['review']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data.loc[index,'review_final'] = str(Final_words)

Split the data into train/test set

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data['review_final'],data['label'],test_size=0.33,random_state=80)
#y_train.dropna(inplace=True)
#y_test.dropna(inplace=True)


In [ ]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

# Word Vectorization

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000, stop_words='english', min_df=.01, max_df=.90)
Tfidf_vect.fit(data['review_final'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)
print(Tfidf_vect.vocabulary_)

## TF-IDF Based on Count Vectorizer

This function will take in a dataset and use countVectorizer to make a df based on word and # of occurrences and also a df based on word and the term weights. I decided to return the 2nd df

In [ ]:
def tf_idf(dataset): 

  countVec = CountVectorizer(max_features= 5000, stop_words='english', min_df=.01, max_df=.90)

  countVec.fit(dataset["review"])
  #useful debug, get an idea of the item list you generated
  #list(countVec.vocabulary_.items())

  countVec_count = countVec.transform(dataset["review"])
  occ = np.asarray(countVec_count.sum(axis=0)).ravel().tolist()

  bowListFrame = pd.DataFrame({'term': countVec.get_feature_names(), 'occurrences': occ})
  #print(bowListFrame)
  #bowListFrame.sort_values(by='occurrences', ascending=False).head(60)

  reviewTransformer = TfidfTransformer()

  #initial fit representation using transformer object
  reviewWeights = reviewTransformer.fit_transform(countVec_count)

  #follow similar process to making new data frame with word occurrences, but with term weights
  reviewWeightsFin = np.asarray(reviewWeights.mean(axis=0)).ravel().tolist()

  #now that we've done Tfid, make a dataframe with weights and names
  reviewWeightFrame = pd.DataFrame({'term': countVec.get_feature_names(), 'weight': reviewWeightsFin})
  #print(reviewWeightFrame)
  reviewWeightFrame=reviewWeightFrame.sort_values(by='weight', ascending=False).head(20)  
  return reviewWeightFrame




In [ ]:
testWeightFrame = tf_idf(test)
trainWeightFrame = tf_idf(train)

print(trainWeightFrame)
print("\n",testWeightFrame)

## TF-IDF Based on Tfidf Vectorizer

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    

In [ ]:
y_train = (train['label']=='neg').astype(int)
y_test = (test['label']=='neg').astype(int)

tfidf = TfidfVectorizer(max_features= 5000, stop_words='english', min_df=.01, max_df=.90)
tfidf.fit(train['review'])
X_train = tfidf.transform(train['review'])
X_test = tfidf.transform(test['review'])

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
yhat = nb.predict(X_test)
print(accuracy_score(y_test, yhat))
print(f1_score(y_test, yhat))
print(precision_score(y_test, yhat))
print(recall_score(y_test, yhat))

In [ ]:
y_prob = nb.predict_proba(X_test)[:,1]
fpr, tpr, thresh = roc_curve(y_test, y_prob)
plot_roc_curve(fpr, tpr)
print(roc_auc_score(y_test, y_prob))

In [ ]:
feats = pd.DataFrame()
feats['words'] = tfidf.get_feature_names()
# Convert log probabilities to probabilities. 
feats['pos'] = np.e**(nb.feature_log_prob_[0, :])
feats['neg'] = np.e**(nb.feature_log_prob_[1, :])
feats.set_index('words', inplace=True)

In [ ]:
feats.sort_values(by='pos',ascending=False).head(15)

In [ ]:
feats.sort_values(by='neg',ascending=False).head(15)

### Which words are most important for distinguishing between pos and neg reviews?

Great, like, story, love, best, people are all words that had a high weight when it came to positive words. 

bad, just, time, plot, acting, and worst are words that scored high according to negative reviews. 

#ML Models

According to our score, SVM performed the best with an accuracy score of 87.19%, followed by NB with an accuracy score of 85%. MLP had the lowest score with 84%.

## Naive Bayes

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,y_train)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

## SVM

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

## Multilayer Perceptron


In [ ]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(solver='lbfgs', alpha=1e-3, random_state=3)
MLP.fit(Train_X_Tfidf,y_train)


predictions_MLP = MLP.predict(Test_X_Tfidf)
print("MLP Accuracy Score -> ",accuracy_score(predictions_MLP, y_test)*100)

# Vader Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [266]:
def sentiment_scores(sentence):
  sid_obj = SentimentIntensityAnalyzer()
  sentiment_dict = sid_obj.polarity_scores(sentence)

  # decide sentiment as positive, negative and neutral 
  if sentiment_dict['compound'] >= 0.05 : 
        return ("pos") 
  
  elif sentiment_dict['compound'] <= - 0.05 : 
        return ("neg") 
  
  else : 
        return ("Neutral") 
  #print("{:-<40} {}".format(sentence, str(sentiment_dict)))

In [ ]:
vader=[]
for i in range(len(data['review'])):
  vader.append(sentiment_scores(data['review'][i]))




# Text Blob

In [ ]:
from textblob import TextBlob

def blobSentiment(text):
  blob = TextBlob(text)
  for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

for i in range(10):
  sentiment_scores(data['review_final'][i])

# Topic Modeling


Run LDA topic modeling using gensim on the movie reviews. How many topics are there? What are
the most common words in each topic?


In [ ]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
ntopics = 8

# Create Dictionary

id2word = corpora.Dictionary(data['review'])

# Create Corpus: Term Document Frequency

corpus = [id2word.doc2bow(text) for text in data.review]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=ntopics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

ldatopics = lda_model.show_topics(formatted=False)
print(lew(lda_model.print_topics()))